# Pythonで推論を実行する
trainingで作成したモデルを用いて、Pythonで推論を実行し、翻訳の精度を評価する

# 推論
Greedyサーチで推論する。

In [1]:
!python generate.py --test_file='../dataset/train.jsonl' --model_file='../training/model/checkpoint_best.pt' --outfile="outfile.txt" --device="cpu" --max_len=100  --search='greedy'

[nltk_data] Downloading package punkt_tab to /Users/utsubo-
[nltk_data]     katsuhiko/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/utsubo-
[nltk_data]     katsuhiko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
^C
Traceback (most recent call last):
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/sample/inference_py/generate.py", line 210, in <module>
    main()
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/sample/inference_py/generate.py", line 206, in main
    KanjiKanaTransformerTest(args).generate()
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/sample/inference_py/generate.py", line 173, in generate
    tgt_token,tgt_prob = self.greedy_decode( transformer,  src, src_mask, max_len=self.args.max_len, start_symbol=BOS_IDX)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# 評価
BlueScoreでGreedyサーチで推論した翻訳結果と，実際の翻訳との比較を行い，評価する。

In [ ]:
# https://thepythoncode.com/article/bleu-score-in-python

from nltk.translate.bleu_score import sentence_bleu, corpus_bleu